In [39]:
import numpy as np
import cv2
import numpy as np
import os
import mediapipe as mp
import time
from tqdm import tqdm

# Create some data

In [94]:
# Name of the folder (Masked, Unmasked, Half_Masked)
name = "Masked"

# Number of pictures to take
datasetSize = 80

if not os.path.exists('Output/faces/' + name):
    os.mkdir('Output/faces/' + name)

# Initialize mediapipe
mp_face_detection = mp.solutions.face_detection

stop = False
count = len(os.listdir('Output/faces/' + name))
datasetSize += count

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:
    
    # define a video capture object
    vid = cv2.VideoCapture(0)
    
    i = 0
    while(not stop):
        # Capture the video frame
        # by frame
        ret, img = vid.read()
    
        img.flags.writeable = False
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cv2.flip(img, 1, img)
        
        # Detect faces
        results = face_detection.process(img)

        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        bounded_img = img.copy()

        try:
            if results.detections:
                # Draw the face detection annotations on the image.
                for detection in results.detections:
                    
                    # Get the bounding box of the face
                    # and its characteristics
                    bb = detection.location_data.relative_bounding_box
                    x = bb.xmin
                    y = bb.ymin
                    h = bb.height
                    w = bb.width

                    camWidth = img.shape[1]
                    camHeight = img.shape[0]

                    # Convert x, y, h and w from percentages to pixels
                    x = int(x * camWidth)
                    y = int(y * camHeight)
                    h = int(h * camHeight)
                    w = int(w * camWidth)
                    
                    # Allows some time to move between the captures
                    # Change the modulo value if you want more or less time between captures
                    i = (i + 1) % 15
                    if i == 0:
                        file_name = "face_" + str(count) + ".jpg"
                        
                        # Resize the image to 250x250px
                        f = cv2.resize(img[y:y+h, x:x+w], (250, 250), interpolation = cv2.INTER_AREA)
                        
                        # Save the capture
                        cv2.imwrite('./Output/faces/' + name + '/' + file_name, f)
                        count += 1
                        
                        # Stop if we reached the wanted dataset size
                        if count >= datasetSize:
                            stop = True
                    
                    # Draw a rectangle to represent the bounding box
                    cv2.rectangle(bounded_img, (x, y),(x + w, y + h), (255, 0, 0), 2)
        except:
            pass

        # Increase output size for ergonomy
        bounded_img = cv2.resize(bounded_img, (0,0), fx=1.5, fy=1.5)
        
        # Display the resulting frame
        cv2.imshow('frame', bounded_img)
        
        # Show the remaining frames to capture before the end
        cv2.setWindowTitle('frame', 'frame : ' + str(abs(count - datasetSize)))
        
        # The 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()

# Train a model

## Load the data

In [122]:
# Define the used directories (same as labels)
label_names = ["Masked", "Unmasked", "Half_Masked"]

# Use it if you want to limit the number of images per category
# to avoid long loading times
# Set it to 0 or less to have no limit
limit = 0

faces = []
labels = []
colors = []

for id, d in enumerate(label_names):
    print("Begin loading images from " + d)
    time.sleep(.5)
    files = os.listdir('Output/faces/' + d)
    
    max_files = min(limit, len(files)) if limit > 0 and limit < len(files) else len(files)
    # Load every image (in the defined limit),
    # resize them to 250x250px
    # and convert them to grayscales
    with tqdm(total=max_files) as pbar:
        for x,f in enumerate(files):
            pbar.update(1)
            if x >= max_files-1:
                break
            faces += [cv2.cvtColor(cv2.resize(cv2.imread('./Output/faces/'+ d + '/' + f), (250, 250), interpolation=cv2.INTER_AREA), cv2.COLOR_BGR2GRAY)]
    #faces += [cv2.cvtColor(cv2.resize(cv2.imread('./Output/faces/'+ d + '/' + f), (250, 250), interpolation=cv2.INTER_AREA), cv2.COLOR_BGR2GRAY) for x,f in enumerate(files) if x < limit and limit > 0]
    
    # Fill the labels according to the index of the directory name
    # Example : 0 for 'Masked', 1 for 'Unmasked' and 2 for 'Half_Masked'
    labels += [id] * (len(faces)-len(labels))

    # add random color in the list
    c = [(np.random.randint(100,255), np.random.randint(100,255), np.random.randint(100,255))]
    while c in colors:
        c = [((np.random.randint(100,255), np.random.randint(100,255), np.random.randint(100,255)))]
    colors += c
    print("Finished loading images from " + d)

Begin loading images from Masked


100%|███████████████████████████████████████████████████████████████████████████████| 444/444 [00:00<00:00, 801.82it/s]


Finished loading images from Masked
Begin loading images from Unmasked


100%|███████████████████████████████████████████████████████████████████████████████| 443/443 [00:00<00:00, 753.41it/s]


Finished loading images from Unmasked
Begin loading images from Half_Masked


100%|███████████████████████████████████████████████████████████████████████████████| 440/440 [00:00<00:00, 809.75it/s]

Finished loading images from Half_Masked


## Reshape the data

In [123]:
with_mask = []
without_mask = []
half_masked = []

with tqdm(total=len(faces)) as pbar:
    for f, l in zip(faces, labels):
        pbar.update(1)
        
        # transform the image to a numpy array
        data = np.array(f)
        # reshape the image to a linear array
        data = data.reshape(1, -1)
        if l == 0:
            with_mask.append(data)
        elif l == 1:
            without_mask.append(data)
        elif l == 2:
            half_masked.append(data)
        else:
            dumbass.append(data)

with_mask = np.array(with_mask).reshape(len(with_mask), -1)
without_mask = np.array(without_mask).reshape(len(without_mask), -1)
half_masked = np.array(half_masked).reshape(len(half_masked), -1)

print(with_mask.shape)
print(without_mask.shape)
print(half_masked.shape)

# concatenate the arrays 
X = np.r_[with_mask, without_mask, half_masked]
print(X.shape)

labels = np.array(labels)
print(labels.shape)

100%|███████████████████████████████████████████████████████████████████████████| 1324/1324 [00:00<00:00, 29396.63it/s]

(443, 62500)
(442, 62500)
(439, 62500)
(1324, 62500)
(1324,)


## Create the model

In [103]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import joblib

### Split the data for train and test

In [124]:
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size=0.25, random_state=42)
print(x_train.shape)

(993, 62500)


### Initialize the model

In [125]:
filename = 'svm_model.sav'

# You can load the model if you want to train over the previous trainings
#svm = joblib.load(filename)
svm = SVC(kernel='rbf', C=4, verbose=True)

### Train it

In [126]:
svm.fit(x_train, y_train)

y_pred = svm.predict(x_test)
print(accuracy_score(y_test, y_pred))

[LibSVM]0.8429003021148036


### Save the model

In [127]:
joblib.dump(svm, filename)

['svm_model.sav']

# Tri it !

In [128]:
mp_face_detection = mp.solutions.face_detection

#svm = joblib.load(filename)

vid = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:
    while(True):
        # Capture the video frame
        # by frame
        ret, img = vid.read()
        cv2.flip(img, 1, img)
        
        img.flags.writeable = False
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        results = face_detection.process(img)

        # Draw the face detection annotations on the image.
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        try:
            if results.detections:
                for detection in results.detections:
                    
                    # Get the bounding box of the face
                    # and its characteristics
                    bb = detection.location_data.relative_bounding_box
                    x = bb.xmin
                    y = bb.ymin
                    h = bb.height
                    w = bb.width

                    camWidth = img.shape[1]
                    camHeight = img.shape[0]

                    # Convert x, y, h and w from percentages to pixels
                    x = int(x * camWidth)
                    y = int(y * camHeight)
                    h = int(h * camHeight)
                    w = int(w * camWidth)
                    
                    # Resize the face capture and grayscale
                    toPredict = cv2.resize(img[y:y+h, x:x+w], (250, 250), interpolation = cv2.INTER_AREA)
                    toPredict = cv2.cvtColor(toPredict, cv2.COLOR_BGR2GRAY)
                    
                    # Reshape the data
                    toPredict = toPredict.reshape(1, -1)

                    # Predict
                    pred = svm.predict(toPredict)
                    pred = int(pred[0])

                    # if the predicted label is in the list of known labels, draw a rectangle around the face and label it
                    if pred < len(label_names):
                        cv2.rectangle(img,(x,y),(x+w,y+h),colors[pred],2)
                        cv2.putText(img, label_names[pred], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, colors[pred], 2)
        except:
            pass

        img = cv2.resize(img, (0,0), fx=1.5, fy=1.5)
        
        # Display the resulting frame
        cv2.imshow('frame', img)
        
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()